In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import time
import random
import sys

In [2]:
proxies = {
    'http': 'http://81.162.195.149',
    'https': 'http://81.162.195.149'
}

In [3]:
actorsDataFrame = pd.read_csv('unique_actors2.csv', encoding = "ISO-8859-1")
actors = actorsDataFrame['actors'].tolist()
len(actors)

3045

In [4]:
chrome_path = "./chromedriver"
options = webdriver.ChromeOptions()
options.add_argument('--load-extension=./adblock4Sellenium/3.34.0_0')
options.add_experimental_option( "prefs",{'profile.managed_default_content_settings.javascript': 2})
# options.add_argument("--user-agent=" + 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36')
driver = webdriver.Chrome(chrome_path, options=options)
                            

In [314]:
actorDataFrame = []

In [315]:
#Finds number of movies from soup
def findNumMovies(soup):
    numCreds = soup.find("div", {"id": "filmo-head-actor"})
    if numCreds is None:
        numCreds = soup.find("div", {"id": "filmo-head-actress"})
    rawNumOfMovies = numCreds.text
    return rawNumOfMovies

In [316]:
def findWinsAndNoms(rawWinsAndNoms, winsAndNoms):
    if ("wins" in rawWinsAndNoms or "win" in rawWinsAndNoms) and ("nominations" in rawWinsAndNoms or "nomination" in rawWinsAndNoms):
        numWins = int(winsAndNoms[0])
        numNoms = int(winsAndNoms[1])
    #if they have only one of the two available
    else:
        if "wins" in rawWinsAndNoms or "win" in rawWinsAndNoms:
            numWins = int(winsAndNoms[0])
            numNoms = numWins
        elif "nominations" in rawWinsAndNoms or "nomination" in rawWinsAndNoms:
            numNoms = int(winsAndNoms[0])
            numWins = 0
    return [numNoms, numWins]

In [317]:
#under: call function to web scrape the page
def scrape(actorName, htmlURL):
        html_doc = requests.get(htmlURL)
        soup = BeautifulSoup(html_doc.text, 'html.parser')
        awards = soup.findAll("span", {"class": "awards-blurb"})
        rawNumOfMovies = findNumMovies(soup)    
    
        rawNumOfMovies = re.findall('\d+', rawNumOfMovies)
        numMovies = int(rawNumOfMovies[0])
        if len(awards) == 0:
            numWins = 0
            numNoms = 0 
        elif len(awards) == 1 :
            #when no oscars and maybe having just a win / nomination on its own
            rawWinsAndNoms = awards[0].text
            winsAndNoms = re.findall('\d+', rawWinsAndNoms)
            #if actor has both wins and nominations
            winsAndNoms = findWinsAndNoms(rawWinsAndNoms, winsAndNoms)
            numNoms = winsAndNoms[0]
            numWins = winsAndNoms[1]
        #When the actor has an oscar ontop of other nominations (is an extra visual they add)
        else:                
            rawWinsAndNoms = awards[1].text
            # winsAndNoms = re.findall('\\s\\d+\\s', rawWinsAndNoms)
            winsAndNoms = re.findall('\d+', rawWinsAndNoms)
            #If they have both wins and nominations available
            winsAndNoms = findWinsAndNoms(rawWinsAndNoms, winsAndNoms)
            numNoms = winsAndNoms[0]
            numWins = winsAndNoms[1]
            #
            specialWins = awards[0].text
            specialWins = re.findall('\d+', specialWins)
            specialWins = int(specialWins[0])
            numWins += specialWins
            numNoms += specialWins
                        
    
    
            # List of the Rating Values (in order; number of wins, num of nominations, and num of movies):
            #actorValues = [numWins, numNoms, numMovies]
        print(actorName, ": Movies: ", numMovies, " Noms: ", numNoms, " Wins: ", numWins)
    
    
        actorDataFrame.append({'actorName': actorName, 'actorMovieCount': numMovies,
                            'actorAwardNominations': numNoms, 'actorAwardWins': numWins})

In [318]:
#TODO: fix when ads come up.
count = len(actors)
    
driver.get('https://www.imdb.com/?ref_=nv_home')
for name in actors:
    waitTime = random.uniform(0.5,1.5)
    time.sleep(waitTime)
    if count % 2 == 0:
        print("number of actors remaining: %i" % count)
    #trying if the actor is searchable
    try:
        #Find search bar, clear and search for actor
        searchBar = driver.find_element_by_id('navbar-query')
        searchBar.clear()
        searchBar.send_keys(name)
        searchBar.send_keys(Keys.RETURN)
        #trying for if people show up at the top of the search
        try:
            #Taking the top actor from the list of names that show up from the search
            try:
                personWanted = driver.find_elements_by_class_name('findResult')
                personURL = personWanted[0].find_element_by_tag_name('a')
                personURL.click()
                
                driver.current_url
                scrape(name, driver.current_url)
            #If top link isnt the actor, iterate over other links in list for actor
            except:
                driver.back()
                personWanted = driver.find_elements_by_class_name('findResult')
                good = False
                person = 1
                #Ensures the person it will be selecting is an actor and not another profession
                while not good:
                    isActor = personWanted[person].find_element_by_tag_name('small').text
                    if "Actress" in isActor or "Actor" in isActor:
                        personURL = personWanted[person].find_element_by_tag_name('a')
                        personURL.click()
                        driver.current_url
                        scrape(name, driver.current_url)
                        good = True
                    person = person + 1

            
        #Except instance when a search of an actor has a non-actor (ex: show) as the top choice
        except Exception as e:
            print('Error on line {}'.format(sys.exc_info()[-1].tb_lineno), type(e).__name__, e)
            #Reason for doing a whole repeat is because of HTML format where cannot determine if link to click on is actor or not
            #Find search bar, clear and search for actor
            searchBar = driver.find_element_by_id('navbar-query')
            searchBar.clear()
            searchBar.send_keys(name)
            searchBar.send_keys(Keys.RETURN)
            assert "No results found." not in driver.page_source
            #Taking the top actor from the list of names that show up from the search
            moreName = driver.find_element_by_partial_link_text('More name matches')
            moreName.click()
            personWanted = driver.find_element_by_class_name('findResult')
            personURL = personWanted.find_element_by_tag_name('a')
            personURL.click()
            driver.current_url
            scrape(name, driver.current_url)
        count -= 1        
    except Exception as e:
        actorDataFrame.append({'actorName': name, 'actorMovieCount': 1,
                            'actorAwardNominations': 0, 'actorAwardWins': 0})
        print('Error on line {}'.format(sys.exc_info()[-1].tb_lineno), type(e).__name__, e)
        print("Actor: ", name," not searchable")
   
actorDataFrame = pd.DataFrame(actorDataFrame, columns = ['actorName', 'actorMovieCount', 'actorAwardNominations', 'actorAwardWins'] )
actorDataFrame.to_csv('actorsFINAL.csv', index = False)
actorDataFrame

number of actors remaining: 3046
Peter Sellers : Movies:  87  Noms:  20  Wins:  12
Patrick Fugit : Movies:  34  Noms:  16  Wins:  4
number of actors remaining: 3044
Max Irons : Movies:  17  Noms:  0  Wins:  0
Jung Suh : Movies:  6  Noms:  2  Wins:  2
number of actors remaining: 3042
Demián Bichir : Movies:  85  Noms:  17  Wins:  9
Tom Hulce : Movies:  31  Noms:  10  Wins:  5
number of actors remaining: 3040
Joseph Kaufmann : Movies:  9  Noms:  0  Wins:  0
Sean Connery : Movies:  93  Noms:  29  Wins:  38
number of actors remaining: 3038
Morris Chestnut : Movies:  51  Noms:  5  Wins:  2
Cate Blanchett : Movies:  70  Noms:  187  Wins:  152
number of actors remaining: 3036
Katie Featherston : Movies:  14  Noms:  2  Wins:  1
Greta Gerwig : Movies:  40  Noms:  141  Wins:  36
number of actors remaining: 3034
Karin Bertling : Movies:  63  Noms:  1  Wins:  1
Takuya Kimura : Movies:  53  Noms:  2  Wins:  3
number of actors remaining: 3032
Error on line 34 IndexError list index out of range
Error

Clifton Collins Jr : Movies:  120  Noms:  11  Wins:  5
Fahadh Faasil : Movies:  48  Noms:  2  Wins:  5
number of actors remaining: 2946
Thomas Jane : Movies:  72  Noms:  8  Wins:  6
Ryan Carnes : Movies:  39  Noms:  0  Wins:  0
number of actors remaining: 2944
Gabrielle Anwar : Movies:  58  Noms:  4  Wins:  0
Jeffrey Combs : Movies:  133  Noms:  5  Wins:  4
number of actors remaining: 2942
Niklas Ulfvarson : Movies:  1  Noms:  0  Wins:  0
Pat Healy : Movies:  106  Noms:  2  Wins:  6
number of actors remaining: 2940
Bodil Jørgensen : Movies:  83  Noms:  15  Wins:  9
Armand Assante : Movies:  136  Noms:  11  Wins:  9
number of actors remaining: 2938
Napoleon Ryan : Movies:  57  Noms:  1  Wins:  1
Simon Yam : Movies:  227  Noms:  18  Wins:  7
number of actors remaining: 2936
Zhang Ziyi : Movies:  38  Noms:  51  Wins:  46
Joe Pesci : Movies:  40  Noms:  14  Wins:  14
number of actors remaining: 2934
Lily Tomlin : Movies:  68  Noms:  72  Wins:  43
Jessica Tandy : Movies:  64  Noms:  17  Win

Jimmy Bennett : Movies:  50  Noms:  5  Wins:  2
Bradford Dillman : Movies:  140  Noms:  3  Wins:  3
number of actors remaining: 2820
Ashton Kutcher : Movies:  36  Noms:  40  Wins:  18
Jon Favreau : Movies:  69  Noms:  11  Wins:  7
number of actors remaining: 2818
Lennie Loftin : Movies:  53  Noms:  0  Wins:  0
Chris Makepeace : Movies:  22  Noms:  1  Wins:  0
number of actors remaining: 2816
Nicholas Hope : Movies:  79  Noms:  2  Wins:  1
Pavel Derevyanko : Movies:  56  Noms:  5  Wins:  1
number of actors remaining: 2814
Timothy Spall : Movies:  140  Noms:  30  Wins:  20
Bono : Movies:  55  Noms:  22  Wins:  10
number of actors remaining: 2812
Method Man : Movies:  73  Noms:  3  Wins:  1
Junichi Okada : Movies:  38  Noms:  2  Wins:  3
number of actors remaining: 2810
Mónica del Carmen : Movies:  26  Noms:  1  Wins:  1
Alan Ladd : Movies:  100  Noms:  1  Wins:  6
number of actors remaining: 2808
Julianne Hough : Movies:  14  Noms:  9  Wins:  3
Stellan Skarsgård : Movies:  142  Noms:  15

number of actors remaining: 2702
Anthony Hopkins : Movies:  134  Noms:  63  Wins:  50
Steve Buscemi : Movies:  159  Noms:  65  Wins:  29
number of actors remaining: 2700
Steven Pasquale : Movies:  24  Noms:  0  Wins:  0
America Ferrera : Movies:  49  Noms:  39  Wins:  21
number of actors remaining: 2698
Fernanda Andrade : Movies:  35  Noms:  0  Wins:  0
Keanu Reeves : Movies:  96  Noms:  22  Wins:  12
number of actors remaining: 2696
Vidya Balan : Movies:  41  Noms:  29  Wins:  38
Lasse Kolsrud : Movies:  23  Noms:  0  Wins:  0
number of actors remaining: 2694
Tobey Maguire : Movies:  47  Noms:  31  Wins:  8
Kip Pardue : Movies:  62  Noms:  1  Wins:  2
number of actors remaining: 2692
Sienna Guillory : Movies:  61  Noms:  2  Wins:  0
Jesper Christensen : Movies:  124  Noms:  12  Wins:  17
number of actors remaining: 2690
Molly Ringwald : Movies:  64  Noms:  7  Wins:  4
Natar Ungalaaq : Movies:  14  Noms:  5  Wins:  5
number of actors remaining: 2688
Jeffrey Wright : Movies:  65  Noms: 

Alex Pettyfer : Movies:  20  Noms:  6  Wins:  4
number of actors remaining: 2582
Aubrey Peeples : Movies:  21  Noms:  0  Wins:  0
Error on line 34 IndexError list index out of range
Error on line 61 AttributeError 'NoneType' object has no attribute 'text'
Actor:  Shereef Akeel  not searchable
Michelle Williams : Movies:  58  Noms:  135  Wins:  44
number of actors remaining: 2580
Anita Björk : Movies:  61  Noms:  1  Wins:  1
Elisha Cuthbert : Movies:  34  Noms:  10  Wins:  2
number of actors remaining: 2578
Roddy Piper : Movies:  159  Noms:  1  Wins:  3
Claudette Colbert : Movies:  79  Noms:  4  Wins:  7
number of actors remaining: 2576
Derek Luke : Movies:  35  Noms:  13  Wins:  7
Eva Santolaria : Movies:  33  Noms:  2  Wins:  2
number of actors remaining: 2574
Giuliano Gemma : Movies:  102  Noms:  2  Wins:  8
Johnny Depp : Movies:  91  Noms:  143  Wins:  79
number of actors remaining: 2572
Kate McNeil : Movies:  62  Noms:  1  Wins:  1
Heike Makatsch : Movies:  63  Noms:  8  Wins:  9
n

...

number of actors remaining: 430
Kay Panabaker : Movies:  42  Noms:  6  Wins:  4
Ronnie Barker : Movies:  75  Noms:  7  Wins:  6
number of actors remaining: 428
Colin Farrell : Movies:  57  Noms:  57  Wins:  14
Charlize Theron : Movies:  59  Noms:  103  Wins:  59
number of actors remaining: 426
Santiago Segura : Movies:  117  Noms:  3  Wins:  8
Jaime Pressly : Movies:  68  Noms:  12  Wins:  6
number of actors remaining: 424
Desmond Harrington : Movies:  42  Noms:  5  Wins:  1
Brett Granstaff : Movies:  12  Noms:  2  Wins:  1
number of actors remaining: 422
Mark Wahlberg : Movies:  65  Noms:  71  Wins:  22
Christopher Abbott : Movies:  32  Noms:  11  Wins:  4
number of actors remaining: 420
Ossie Davis : Movies:  107  Noms:  15  Wins:  13
Anne Hathaway : Movies:  44  Noms:  106  Wins:  65
number of actors remaining: 418
Jerry Springer : Movies:  35  Noms:  2  Wins:  2
Jonny Lee Miller : Movies:  52  Noms:  12  Wins:  1
number of actors remaining: 416
David Rasche : Movies:  129  Noms:  2

number of actors remaining: 320
Rufus Sewell : Movies:  63  Noms:  6  Wins:  0
Guillaume Canet : Movies:  67  Noms:  16  Wins:  10
number of actors remaining: 318
Samira Maas : Movies:  8  Noms:  1  Wins:  1
Tommy Lee Jones : Movies:  83  Noms:  90  Wins:  43
number of actors remaining: 316
Grégoire Ludig : Movies:  29  Noms:  0  Wins:  0
Scott Adkins : Movies:  59  Noms:  1  Wins:  1
number of actors remaining: 314
Andrew Robinson : Movies:  108  Noms:  5  Wins:  1
Gérard Depardieu : Movies:  235  Noms:  34  Wins:  24
number of actors remaining: 312
Gina Lollobrigida : Movies:  68  Noms:  7  Wins:  32
Naomi Watts : Movies:  85  Noms:  88  Wins:  57
number of actors remaining: 310
Bruce Willis : Movies:  119  Noms:  38  Wins:  22
Zoe Sloane : Movies:  11  Noms:  0  Wins:  0
number of actors remaining: 308
Donald Faison : Movies:  84  Noms:  14  Wins:  2
Darsheel Safary : Movies:  6  Noms:  2  Wins:  2
number of actors remaining: 306
Elizabeth Banks : Movies:  87  Noms:  35  Wins:  16
I

number of actors remaining: 194
Lewis J. Stadlen : Movies:  30  Noms:  0  Wins:  0
Denis Nikiforov : Movies:  28  Noms:  4  Wins:  1
number of actors remaining: 192
Joe Alwyn : Movies:  7  Noms:  1  Wins:  1
John Lone : Movies:  24  Noms:  3  Wins:  3
number of actors remaining: 190
Til Schweiger : Movies:  75  Noms:  6  Wins:  26
John Ireland : Movies:  201  Noms:  3  Wins:  3
number of actors remaining: 188
Huang Bo : Movies:  37  Noms:  16  Wins:  9
Camilla Belle : Movies:  40  Noms:  5  Wins:  0
number of actors remaining: 186
Brendan Fraser : Movies:  70  Noms:  11  Wins:  8
Oksana Akinshina : Movies:  31  Noms:  7  Wins:  6
number of actors remaining: 184
Frances Conroy : Movies:  100  Noms:  24  Wins:  4
Rick Malambri : Movies:  16  Noms:  0  Wins:  0
number of actors remaining: 182
Louise Grinberg : Movies:  15  Noms:  0  Wins:  0
Blake Woodruff : Movies:  9  Noms:  1  Wins:  1
number of actors remaining: 180
Rica Matsumoto : Movies:  105  Noms:  0  Wins:  0
Nick Frost : Movies

number of actors remaining: 70
Steve Harvey : Movies:  12  Noms:  18  Wins:  18
Howard Stern : Movies:  5  Noms:  3  Wins:  3
number of actors remaining: 68
Chiwetel Ejiofor : Movies:  50  Noms:  75  Wins:  54
Viola Savoy : Movies:  2  Noms:  0  Wins:  0
number of actors remaining: 66
John Jarratt : Movies:  86  Noms:  5  Wins:  2
Suriya  : Movies:  47  Noms:  18  Wins:  9
number of actors remaining: 64
Tom Hardy : Movies:  55  Noms:  77  Wins:  17
Hans Matheson : Movies:  30  Noms:  0  Wins:  0
number of actors remaining: 62
Wil Wheaton : Movies:  126  Noms:  5  Wins:  6
Andy Lau : Movies:  170  Noms:  42  Wins:  18
number of actors remaining: 60
Sophie Marceau : Movies:  45  Noms:  5  Wins:  8
Jonathon Trent : Movies:  32  Noms:  0  Wins:  0
number of actors remaining: 58
Danny Elfman : Movies:  12  Noms:  100  Wins:  80
Sandy Duncan : Movies:  35  Noms:  4  Wins:  2
number of actors remaining: 56
Bruno Corazzari : Movies:  99  Noms:  0  Wins:  0
David Arquette : Movies:  124  Noms: 

,actorName,actorMovieCount,actorAwardNominations,actorAwardWins
0,Peter Sellers,87,20,12
1,Patrick Fugit,34,16,4
2,Max Irons,17,0,0
3,Jung Suh,6,2,2
4,Demián Bichir,85,17,9
5,Tom Hulce,31,10,5
6,Joseph Kaufmann,9,0,0
7,Sean Connery,93,29,38
8,Morris Chestnut,51,5,2
9,Cate Blanchett,70,187,152
